In [1]:
import numpy as np
from sklearn.datasets import load_digits
import pandas as pd

In [2]:
mnist = load_digits()
X,y = mnist.data, mnist.target

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
y_odd = ((y == 2) | (y == 4) | (y == 6) | (y == 8) | (y == 0))

In [5]:
y_odd = y_odd.reshape(y_odd.shape[0],1).astype(int)

In [6]:
y_odd.shape

(1797, 1)

In [7]:
X_tr,X_te,y_tr, y_te = train_test_split(X,y_odd,test_size=0.3, random_state=40)

In [29]:
X_tr = X_tr.astype('np.float128')
X_te = X_te.astype('np.float128')
y_tr = y_tr.astype('np.float128')
y_te = y_te.astype('np.float128')

TypeError: data type 'np.float128' not understood

In [9]:
def initializer(X,y):
    n_x = X.shape[0]
    n_h = 5
    n_y = y.shape[0]

    W1 = np.random.randn(n_h,n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y,n_h) * 0.01
    b2 = np.zeros((n_y,1))
    
    return {"size":[n_x,n_h,n_y],
            "params":[W1, b1, W2, b2]}

In [10]:
W1, b1, W2, b2 = initializer(X_tr, y_tr)["params"]

In [11]:
def ReLU(z):
    relu = np.maximum(0,z)
    return relu

def ReLU_prime(z):
    result = (z >= 0).astype(int)
    return result

def sigmoid(z):
    s = 1.0/(1+np.exp(-z))
    return s

In [12]:
ReLU([1,2,3,4,5,6,-5])

array([1, 2, 3, 4, 5, 6, 0])

In [13]:
def fprop(X, parameters):
    
    W1 = parameters[0]
    b1 = parameters[1]
    W2 = parameters[2]
    b2 = parameters[3]
    
    Z1 = np.dot(W1,X)+b1
    A1 = ReLU(Z1)
    Z2 = np.dot(W2,A1)+b2
    A2 = sigmoid(Z2)

    #assert(A2.shape == (size[2], X.shape[1]))
    
    
    return [Z1,A1,Z2,A2]

In [14]:
Z1,A1,Z2,A2 = fprop(X_tr, [W1, b1, W2, b2])

In [15]:
def cost(A2, y):
    m = y.shape[0]

    cost  = (-1/m) * np.sum((np.multiply(y , np.log(A2))) + (np.multiply((1-y) , np.log(1-A2))))
    J = float(np.squeeze(cost))
    
    return J

In [16]:
cost(A2, y_tr)

44.36640999124313

In [17]:
def backward_propagation(parameters, fprop, X, y):
    m = X.shape[1]
    
    W1 = parameters[0]
    b1 = parameters[1]
    W2 = parameters[2]
    b2 = parameters[3]

    Z1 = fprop[0]
    A1 = fprop[1]
    Z2 = fprop[2]
    A2 = fprop[3]

    dZ2 = A2 - y
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis = 1, keepdims=True)
    dZ1 = np.dot(W2.T, dZ2) *  (ReLU_prime(Z1))
    dW1 = (1/m) * np.dot(dZ1,X.T)
    db1 = (1/m) * np.sum(dZ1, axis = 1, keepdims=True)

    grads = {
        "dW1": dW1,
        "db1": db1,
        "dZ1": dZ1,
        "dW2": dW2,
        "db2": db2,
        "dZ2": dZ2,    
    }
    return grads

In [18]:
grads = backward_propagation([W1, b1, W2, b2], [Z1,A1,Z2,A2], X_tr, y_tr)

In [19]:
def update_parameters(parameters, grad, learning_rate):

    W1 = parameters[0]
    b1 = parameters[1]
    W2 = parameters[2]
    b2 = parameters[3]

    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]

    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2

    parameters = [W1, b1, W2, b2]
    
    return parameters

In [24]:
update_parameters([W1, b1, W2, b2],grads,0.1)

[array([[-0.0026766 ,  0.00032985,  0.00729427, ...,  0.01572954,
          0.00732482,  0.01706953],
        [-0.02374799, -0.02653466, -0.02728272, ..., -0.03791729,
         -0.03373308, -0.03617991],
        [-0.06712413, -0.0431784 , -0.03764211, ..., -0.04366167,
         -0.05281877, -0.07402739],
        [-0.01676365, -0.01162049, -0.00227327, ...,  0.00796245,
         -0.00345037, -0.03321419],
        [ 0.01180146, -0.00469875,  0.00124898, ...,  0.0158736 ,
          0.0199922 ,  0.01745109]]),
 array([[ 0.00731432],
        [-0.00321975],
        [-0.008631  ],
        [-0.00299823],
        [ 0.00476812]]),
 array([[ 0.00905607, -0.11415923, -0.03134378, -0.02604681, -0.04883148],
        [ 0.02597411,  0.10480524,  0.03681406,  0.00289539,  0.05052703],
        [-0.0081372 ,  0.12821997,  0.02423891,  0.0016897 ,  0.06629646],
        ...,
        [-0.00110629, -0.10863283, -0.04193858, -0.02073008, -0.04990824],
        [-0.00693964, -0.10984574, -0.02641288, -0.0159192

In [25]:
def nn_model(X, y, learning_rate= 0.1,num_iter = 10001, print_cost= True):
    np.random.seed(3)

    n_x , n_fl , n_y = initializer(X,y)["size"]

    parameters = initializer(X,y)["params"]

    W1 = parameters[0]
    b1 = parameters[1]
    W2 = parameters[2]
    b2 = parameters[3]

    for i in range(num_iter):
        #print(parameters,"****")

        # fp
        Z1,A1,Z2,A2 = fprop(X,parameters)
        
        # cost
        costf = cost(A2, y)
        
        # bp
        grads = backward_propagation(parameters,[Z1,A1,Z2,A2],X,y)
        
        # gd
        parameters = update_parameters(parameters, grads,learning_rate)


        # print cost
        if print_cost and i%1000 == 0:
            print ("Cost after iteration %i: %f" %(i, costf))
            #print(i,"*****",A2,"****")
            
    return parameters

In [26]:
for h in [0.001,0.01,1,10,100]:
    print("h = ", h)
    p = nn_model(X_tr,y_tr,h, num_iter = 2001)
    p

h =  0.001
Cost after iteration 0: 44.393496


C:\Windows\Temp\ipykernel_11356\3408913607.py:4: RuntimeWarning: divide by zero encountered in log
  cost  = (-1/m) * np.sum((np.multiply(y , np.log(A2))) + (np.multiply((1-y) , np.log(1-A2))))
C:\Windows\Temp\ipykernel_11356\3408913607.py:4: RuntimeWarning: invalid value encountered in multiply
  cost  = (-1/m) * np.sum((np.multiply(y , np.log(A2))) + (np.multiply((1-y) , np.log(1-A2))))
C:\Windows\Temp\ipykernel_11356\725635873.py:10: RuntimeWarning: overflow encountered in exp
  s = 1.0/(1+np.exp(-z))


Cost after iteration 1000: nan
Cost after iteration 2000: nan
h =  0.01
Cost after iteration 0: 44.393496
Cost after iteration 1000: nan
Cost after iteration 2000: nan
h =  1
Cost after iteration 0: 44.393496
Cost after iteration 1000: nan


KeyboardInterrupt: 

In [ ]:
p = nn_model(X_tr,y_tr, num_iter = 10000, learning_rate = 1000)


In [ ]:
def predict(parameters, X,y):
    
    Z1,A1,Z2,A2 = fprop(X,y)

    predictions = (A2>0.5)

    return predictions

In [ ]:
preds = predict(p,X_te,y_te)

In [ ]:
preds

In [ ]:
#print('Accuracy: %d' %float((np.dot(y_te,preds.T) + np.dot(1 - y_te, 1 - preds.T))/float(y_te.size)*100)+'%')

In [ ]:
Accuracy = (preds == y_te)

In [ ]:
Accuracy.sum()/(Accuracy.shape[0]*Accuracy.shape[1])